In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None
raw_df = pd.read_csv("data/match_map_stats.csv")

In [2]:
match_ids = raw_df.match_id.unique()
len(match_ids)

642

In [3]:
#The first 6 features of the dataframe can be easily set:
team_one_list = []
team_two_list = []
date_list = []
winner_list = []
stage_list = []

for m in match_ids:
    temp_df = raw_df[raw_df['match_id'] == m]
    team_one_list.append(temp_df.team_one_name.unique()[0])
    team_two_list.append(temp_df.team_two_name.unique()[0])
    date_list.append(temp_df.round_start_time.unique()[0])
    winner_list.append(temp_df.match_winner.unique()[0])
    stage_list.append(temp_df.stage.unique()[0])
    
#Let's zip this all up and make a dataframe!
zipped_data = list(zip(match_ids, team_one_list, team_two_list, date_list, stage_list, winner_list))
columns = ['id', 'team_one', 'team_two', 'date', 'stage', 'winner']    

df = pd.DataFrame(zipped_data, columns=columns)

#Make sure the date column is in the proper format
df['date']=pd.to_datetime(df['date'])

In [4]:
#Set the corona_virus_isolation feature.  This will be True if games are being played while in isolation because of the pandemic
df['corona_virus_isolation'] = False

#We will use a mask to properly set the feature:
mask = (df['date'] > '03/20/2020')
df['corona_virus_isolation'] = mask

In [5]:
#Initiate some features:
df['t1_wins_season'] = ""
df['t1_losses_season'] = ""
df['t2_wins_season'] = ""
df['t2_losses_season'] = ""
df['t1_matches_season'] = ""
df['t2_matches_season'] = ""
df['t1_win_percent_season'] = ""
df['t2_win_percent_season'] = ""
df['t1_wins_alltime'] = ""
df['t1_losses_alltime'] = ""
df['t2_wins_alltime'] = ""
df['t2_losses_alltime'] = ""
df['t1_matches_alltime'] = ""
df['t2_matches_alltime'] = ""
df['t1_win_percent_alltime'] = ""
df['t2_win_percent_alltime'] = ""
df['t1_wins_last_3'] = ""
df['t1_losses_last_3'] = ""
df['t2_wins_last_3'] = ""
df['t2_losses_last_3'] = ""
df['t1_win_percent_last_3'] = ""
df['t2_win_percent_last_3'] = ""
df['t1_wins_last_5'] = ""
df['t1_losses_last_5'] = ""
df['t2_wins_last_5'] = ""
df['t2_losses_last_5'] = ""
df['t1_win_percent_last_5'] = ""
df['t2_win_percent_last_5'] = ""
df['t1_wins_last_10'] = ""
df['t1_losses_last_10'] = ""
df['t2_wins_last_10'] = ""
df['t2_losses_last_10'] = ""
df['t1_win_percent_last_10'] = ""
df['t2_win_percent_last_10'] = ""
df['t1_wins_last_20'] = ""
df['t1_losses_last_20'] = ""
df['t2_wins_last_20'] = ""
df['t2_losses_last_20'] = ""
df['t1_win_percent_last_20'] = ""
df['t2_win_percent_last_20'] = ""





In [6]:
#Returned {team_one_wins, team_one_losses, team_one_total_matches, team_two_wins, team_two_losses, team_two_total_matches}
def get_win_loss_total(wl_df, wl_id):
    t1 = ((wl_df[wl_df['id'] == wl_id]).team_one).iloc[0]
    t2 = ((wl_df[wl_df['id'] == wl_id]).team_two).iloc[0]
    date = ((wl_df[wl_df['id'] == wl_id]).date).iloc[0]
    #print(f"Team 1: {t1} Team 2: {t2} Date: {date}")
    teams=[t1, t2]
    
    return_info=[]
    
    for team in teams:
        df_partial_season=wl_df[wl_df['date']<date]
        season_1 = df_partial_season[df_partial_season['team_one'] == team]
        season_2 = df_partial_season[df_partial_season['team_two'] == team]
        team_season = season_1.append(season_2)
        wins = len(team_season[team_season['winner'] == team])
        losses = len(team_season[team_season['winner'] != team])
        matches = wins+losses
        #print(f"{team} {len(season_1)} {len(season_2)} {len(team_season)} {wins} ")
        return_info.append(wins)
        return_info.append(losses)
        return_info.append(matches)
        
    return(return_info)


In [7]:
def find_team_match_results(master_df, min_date, max_date, type):
    df_reduced = master_df[master_df['date'] > min_date]
    df_reduced = df_reduced[df_reduced['date'] < max_date]
    
    #We can use this to make an ID list...
    id_list = df_reduced.id.unique()
    
    for i in id_list:
        win_loss_list = (get_win_loss_total(df_reduced, i))
        mask = master_df['id'] == i

        #calculate the win percentages for each match
        if win_loss_list[2] == 0:
            t1_win_percent = 0
        else:
            t1_win_percent = win_loss_list[0] / win_loss_list[2]
        if win_loss_list[5] == 0:
            t2_win_percent = 0
        else:
            t2_win_percent = win_loss_list[3] / win_loss_list[5]
        
        
        if type=='2020_season':
            master_df['t1_wins_season'][mask] = win_loss_list[0]
            master_df['t1_losses_season'][mask] = win_loss_list[1]
            master_df['t1_matches_season'][mask] = win_loss_list[2]
            master_df['t1_win_percent_season'][mask] = t1_win_percent
            master_df['t2_wins_season'][mask] = win_loss_list[3]
            master_df['t2_losses_season'][mask] = win_loss_list[4]
            master_df['t2_matches_season'][mask] = win_loss_list[5]  
            master_df['t2_win_percent_season'][mask] = t2_win_percent
        if type=='all_time':
            master_df['t1_wins_alltime'][mask] = win_loss_list[0]
            master_df['t1_losses_alltime'][mask] = win_loss_list[1]
            master_df['t1_matches_alltime'][mask] = win_loss_list[2]
            master_df['t1_win_percent_alltime'][mask] = t1_win_percent
            master_df['t2_wins_alltime'][mask] = win_loss_list[3]
            master_df['t2_losses_alltime'][mask] = win_loss_list[4]
            master_df['t2_matches_alltime'][mask] = win_loss_list[5]  
            master_df['t2_win_percent_alltime'][mask] = t2_win_percent
            
    
                
    return(master_df)

In [8]:
def find_last_n_results(master_df, n):
    id_list = master_df.id.unique()
    for i in id_list:
        t1 = ((master_df[master_df['id'] == i]).team_one).iloc[0]
        t2 = ((master_df[master_df['id'] == i]).team_two).iloc[0]
        date = ((master_df[master_df['id'] == i]).date).iloc[0]

        teams=[t1, t2]
    
        win_loss_list=[]

        for team in teams:

            df_partial_season=master_df[master_df['date']<date]
            season_1 = df_partial_season[df_partial_season['team_one'] == team]
            season_2 = df_partial_season[df_partial_season['team_two'] == team]
                        
            #We need to combine season 1 and season 2 and sort by date
            team_df = pd.concat([season_1, season_2])
            #print(f"season_1: {season_1.shape} season_2: {season_2.shape} team_df: {team_df.shape}")
            team_df.sort_values(by=['date'], inplace=True, ascending=False)
            #display(team_df.head)
            top_n_team_df = team_df.head(n)
            
            wins = len(top_n_team_df[top_n_team_df['winner'] == team])
            losses = len(top_n_team_df[top_n_team_df['winner'] != team])
            matches = wins+losses
            #print(f"{team} {len(season_1)} {len(season_2)} {len(team_season)} {wins} ")
            if matches >= n:
                win_loss_list.append(wins)
                win_loss_list.append(losses)
                win_loss_list.append(matches)
            else:
                win_loss_list.append("")
                win_loss_list.append("")
                win_loss_list.append("")
            
            
        
        #print(return_info)
    
        mask = master_df['id'] == i

        #calculate the win percentages for each match
        if win_loss_list[2] == 0:
            t1_win_percent = 0
        elif win_loss_list[2] == "":
            t1_win_percent = ""
        else:
            t1_win_percent = win_loss_list[0] / win_loss_list[2]
        if win_loss_list[5] == 0:
            t2_win_percent = 0
        elif win_loss_list[5] == "":
            t2_win_percent = ""
        else:
            t2_win_percent = win_loss_list[3] / win_loss_list[5]
        
        if n == 3:
            master_df['t1_wins_last_3'][mask] = win_loss_list[0]
            master_df['t1_losses_last_3'][mask] = win_loss_list[1]
            master_df['t1_win_percent_last_3'][mask] = t1_win_percent
            master_df['t2_wins_last_3'][mask] = win_loss_list[3]
            master_df['t2_losses_last_3'][mask] = win_loss_list[4]
            master_df['t2_win_percent_last_3'][mask] = t2_win_percent
        if n == 5:
            master_df['t1_wins_last_5'][mask] = win_loss_list[0]
            master_df['t1_losses_last_5'][mask] = win_loss_list[1]
            master_df['t1_win_percent_last_5'][mask] = t1_win_percent
            master_df['t2_wins_last_5'][mask] = win_loss_list[3]
            master_df['t2_losses_last_5'][mask] = win_loss_list[4]
            master_df['t2_win_percent_last_5'][mask] = t2_win_percent
        elif n == 10:
            master_df['t1_wins_last_10'][mask] = win_loss_list[0]
            master_df['t1_losses_last_10'][mask] = win_loss_list[1]
            master_df['t1_win_percent_last_10'][mask] = t1_win_percent
            master_df['t2_wins_last_10'][mask] = win_loss_list[3]
            master_df['t2_losses_last_10'][mask] = win_loss_list[4]
            master_df['t2_win_percent_last_10'][mask] = t2_win_percent
        elif n == 20:
            master_df['t1_wins_last_20'][mask] = win_loss_list[0]
            master_df['t1_losses_last_20'][mask] = win_loss_list[1]
            master_df['t1_win_percent_last_20'][mask] = t1_win_percent
            master_df['t2_wins_last_20'][mask] = win_loss_list[3]
            master_df['t2_losses_last_20'][mask] = win_loss_list[4]
            master_df['t2_win_percent_last_20'][mask] = t2_win_percent    
    
    return master_df

In [9]:
results_2020 = find_team_match_results(df, '01/01/2020', '12/31/2020', '2020_season')
results_alltime = find_team_match_results(results_2020, '11/25/2010', '01/01/2080', 'all_time')


In [10]:
results_last_5 = find_last_n_results(results_alltime, 5)

In [11]:
results_last_10 = find_last_n_results(results_last_5, 10)

In [12]:
results_last_3 = find_last_n_results(results_last_10, 3)

In [13]:
results_last_20 = find_last_n_results(results_last_3, 20)

In [14]:
results_last_20.to_csv('test3.csv')